<a href="https://colab.research.google.com/github/siiddd/Risk-Analysis-Chatbot/blob/main/Final_ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [61]:
import pandas as pd
import numpy as np
import os
import re
import nltk

In [8]:
nltk.download('popular')

[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/cmudict.zip.
[nltk_data]    | Downloading package gazetteers to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/gazetteers.zip.
[nltk_data]    | Downloading package genesis to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/genesis.zip.
[nltk_data]    | Downloading package gutenberg to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/gutenberg.zip.
[nltk_data]    | Downloading package inaugural to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/inaugural.zip.
[nltk_data]    | Downloading package movie_reviews to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping corpora/movie_reviews.zip.
[nltk_data]    | Downloading package names to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/names.zip.
[nltk_data]    | Downloading package shakespeare to /root/nltk_data...
[nlt

True

In [10]:
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

In [22]:
dataset = pd.read_csv('https://raw.githubusercontent.com/siiddd/Risk-Analysis-Chatbot/main/dataset_augmented.csv')
dataset = dataset.iloc[:, [-2, -1]]
dataset.head(3)

,Description,Accident Level
0,at level access scissor performed the sustaini...,0
1,in general store while two store attendants re...,0
2,being p m approximately in the nv in the tecno...,3


In [23]:
description = dataset.iloc[:, -2]
description.head(3)

0    at level access scissor performed the sustaini...
1    in general store while two store attendants re...
2    being p m approximately in the nv in the tecno...
Name: Description, dtype: object

In [40]:
words = []

for i in range(len(description)):
  dataset_word = nltk.word_tokenize(description[i])
  words = [*words, *dataset_word]
  del dataset_word

In [48]:
words = list(set(words))
len(words)

4657

In [49]:
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip glove.6B.zip
!rm glove.6B.zip glove.6B.300d.txt

--2022-03-23 12:29:52--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2022-03-23 12:29:53--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2022-03-23 12:29:53--  http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glove.6

In [56]:
os.getcwd()

'/content'

In [66]:
embeddings = {}

f = open('glove.6B.200d.txt', 'r', encoding = 'utf-8')
for line in f:
  values = line.split()
  words = values[0]
  coefs = np.asarray(values[1:], dtype = 'float32')
  embeddings[words] = coefs
f.close()

In [76]:
data_list = list()
for i in range(len(description)):
    sentence = np.zeros(200)
    count = 0
    dataset_word = nltk.word_tokenize(description[i])
    for x in dataset_word:
        try:
            sentence += embeddings[x]
            count += 1
        except KeyError:
            continue
    data_list.append(sentence / count)

In [83]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

model =RandomForestClassifier()

model.fit(data_list, dataset['Accident Level'])

pred = model.predict(data_list)

print('Random Forest \nTraining accuracy', accuracy_score(dataset['Accident Level'], pred))

Random Forest 
Training accuracy 0.9983844911147012


In [89]:
import xgboost as xgb

model = xgb.XGBClassifier()

model.fit(pd.DataFrame(data_list), dataset['Accident Level'])

pred = model.predict(pd.DataFrame(data_list))

print('XGB Classifier: \nTraining accuracy', accuracy_score(dataset['Accident Level'], pred))

XGB Classifier: 
Training accuracy 0.9959612277867528


In [90]:
from sklearn.svm import SVC

model = SVC(kernel = 'rbf')

model.fit(data_list, dataset['Accident Level'])

pred = model.predict(data_list)

print('Support Vector Machine \nTraining accuracy', accuracy_score(dataset['Accident Level'], pred))

Support Vector Machine 
Training accuracy 0.6631663974151858
